In [1]:
import os
import pickle
import pandas as pd
from llama_cpp import Llama
DATA_PATH = os.environ.get('dataset_filepath')
BUGGY_DATAPATH = DATA_PATH+"/buggy_dataset"
STABLE_DATAPATH = DATA_PATH+"/stable_dataset"
bugfixes_test_path = BUGGY_DATAPATH+"/bugfixes_test.pickle"
bugfixes_train_path = BUGGY_DATAPATH+"/bugfixes_train.pickle"
bugfixes_valid_path = BUGGY_DATAPATH+"/bugfixes_valid.pickle"

stable_code_test_path = STABLE_DATAPATH+"/stable_code_test.pickle"
stable_code_train_path = STABLE_DATAPATH+"/stable_code_train.pickle"
stable_code_valid_path = STABLE_DATAPATH+"/stable_code_valid.pickle"


In [2]:
# read files and convert to dataframes
bugfixes_test = pd.read_pickle(bugfixes_test_path)
bugfixes_train = pd.read_pickle(bugfixes_train_path)
bugfixes_valid = pd.read_pickle(bugfixes_valid_path)

stable_code_test = pd.read_pickle(stable_code_test_path)
stable_code_train = pd.read_pickle(stable_code_train_path)
stable_code_valid = pd.read_pickle(stable_code_valid_path)


In [7]:
yi_coder = Llama.from_pretrained(repo_id="bartowski/Llama-3.2-1B-Instruct-GGUF",
                                 filename="Llama-3.2-1B-Instruct-Q3_K_XL.gguf",
                                 verbose=False)


Llama-3.2-1B-Instruct-Q3_K_XL.gguf:   0%|          | 0.00/796M [00:00<?, ?B/s]

llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_f32_f32                    (not supported)
ggml_metal_init: skipping kernel_mul_mm_f16_f32                    (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_q4_0_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_q4

In [3]:
sample_data = bugfixes_train.sample(1,ignore_index=True)
sample_data

,after_merge,before_merge,filename,full_file_code_after_merge,full_file_code_before_merge,function_name,url,source code and errors,full_traceback,traceback_type,before_merge_without_docstrings,after_merge_without_docstrings,before_merge_docstrings,after_merge_docstrings,path_to_snippet_before_merge,path_to_snippet_after_merge
0,"def initialize(self, orderbook):\n ...","def initialize(self, orderbook):\n ...",jmclient/jmclient/taker.py,#! /usr/bin/env python\nfrom __future__ import...,#! /usr/bin/env python\nfrom __future__ import...,Taker.initialize,https://github.com/JoinMarket-Org/joinmarket-c...,"[{'piece_type': 'error message', 'piece_conten...","2018-07-21 00:03:59,900 [MainThread ] [INFO ]...",exceptions.AttributeError,"def initialize(self, orderbook):\n ...","def initialize(self, orderbook):\n ...",[],[],buggy_snippets_files/489ccbbdbb903072aa91cef01...,buggy_snippets_files/489ccbbdbb903072aa91cef01...


In [4]:
print("Before Merge")
print(sample_data.loc[0,'before_merge'])
print("############################################")
print("After Merge")
print(sample_data.loc[0,'after_merge'])
print("############################################")
print("Source Code and Errors")
print(sample_data.loc[0,'source code and errors'])


Before Merge
    def initialize(self, orderbook):
        """Once the daemon is active and has returned the current orderbook,
        select offers, re-initialize variables and prepare a commitment,
        then send it to the protocol to fill offers.
        """
        if self.aborted:
            return (False,)
        self.taker_info_callback("INFO", "Received offers from joinmarket pit")
        #choose the next item in the schedule
        self.schedule_index += 1
        if self.schedule_index == len(self.schedule):
            self.taker_info_callback("INFO", "Finished all scheduled transactions")
            self.on_finished_callback(True)
            return (False,)
        else:
            #read the settings from the schedule entry
            si = self.schedule[self.schedule_index]
            self.mixdepth = si[0]
            self.cjamount = si[1]
            #non-integer coinjoin amounts are treated as fractions
            #this is currently used by the tumbler algo
 

In [49]:
print(sample_data.loc[0,'after_merge'])

def get_params():
    parser = argparse.ArgumentParser()
    # Data path
    parser.add_argument('--datastore', type=str, dest='datastore', help="Datastore path")
    parser.add_argument('--train-datapath', type=str, dest='train_datapath')
    parser.add_argument('--validation-datapath', type=str, dest='validation_datapath')
    parser.add_argument('--surprise-reader', type=str, dest='surprise_reader')
    parser.add_argument('--usercol', type=str, dest='usercol', default='userID')
    parser.add_argument('--itemcol', type=str, dest='itemcol', default='itemID')
    # Metrics
    parser.add_argument('--rating-metrics', type=str, nargs='*', dest='rating_metrics', default=[])
    parser.add_argument('--ranking-metrics', type=str, nargs='*', dest='ranking_metrics', default=[])
    parser.add_argument('--k', type=int, dest='k', default=None)
    parser.add_argument('--remove-seen', dest='remove_seen', action='store_false')
    # Training parameters
    parser.add_argument('--random-state', 

In [12]:
easy_test = """def average(a,b):
    return (a+b)/2
"""

In [43]:
resp = yi_coder(
    f"""
    Does the code below contain any bugs? If so, what are they?
    <code>
    {sample_data.loc[0,'before_merge']}
    </code>
    """, 
    # stop=["Q:"],  # Stop at the first period or newline
    max_tokens=1024,      # Adjust token limit based on your needs
    temperature=0.1,  # Adjust temperature based on your needs
    top_p=0.9,        # Adjust nucleus sampling based on your needs
)
resp

{'id': 'cmpl-70d067c9-7fda-4b60-bc1b-049a39f2f00a',
 'object': 'text_completion',
 'created': 1737831350,
 'model': '/Users/zanchitta/.cache/huggingface/hub/models--bartowski--Llama-3.2-1B-Instruct-GGUF/snapshots/067b946cf014b7c697f3654f621d577a3e3afd1c/./Llama-3.2-1B-Instruct-Q3_K_XL.gguf',
 'choices': [{'text': '\n\n\n\n    In the above code, the following bugs are present:\n\n1.  The `bin_grads_and_vars` and `fp_grads_and_vars` lists are not correctly defined. The `is_binary` method is not defined anywhere in the code. It seems like it should be a method of the `BinaryGradient` class.\n2.  The `fp_train_op` is not defined anywhere in the code. It seems like it should be a method of the `FPGradient` class.\n3.  The `name` parameter is not defined anywhere in the code. It seems like it should be a parameter of the `apply_gradients` method.\n4.  The `name` parameter is not defined in the `apply_gradients` method. It seems like it should be a parameter of the `apply_gradients` method.\n

In [44]:
print(resp['choices'][0]['text'])





    In the above code, the following bugs are present:

1.  The `bin_grads_and_vars` and `fp_grads_and_vars` lists are not correctly defined. The `is_binary` method is not defined anywhere in the code. It seems like it should be a method of the `BinaryGradient` class.
2.  The `fp_train_op` is not defined anywhere in the code. It seems like it should be a method of the `FPGradient` class.
3.  The `name` parameter is not defined anywhere in the code. It seems like it should be a parameter of the `apply_gradients` method.
4.  The `name` parameter is not defined in the `apply_gradients` method. It seems like it should be a parameter of the `apply_gradients` method.
5.  The `name` parameter is not defined in the `apply_gradients` method. It seems like it should be a parameter of the `apply_gradients` method.
6.  The `name` parameter is not defined in the `apply_gradients` method. It seems like it should be a parameter of the `apply_gradients` method.
7.  The `name` parameter is not defi